##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Entrenamiento y servicio de Vertex AI con TFX y Vertex Pipelines


<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
<td><a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?q=download_url%3Dhttps://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Ejecutar en Google Cloud Vertex AI Workbench</a></td>
</table></div>


Este tutorial basado en un bloc de notas creará y ejecutará una canalización de TFX que entrena un modelo de aprendizaje automático (ML) a través del servicio Vertex AI Training y lo publica en Vertex AI para su servicio.

Este bloc de notas se basa en la canalización de TFX que compilamos en el [Tutorial de canalizaciones simples de TFX para Vertex Pipelines](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple). Si aún no ha leído ese tutorial, debe leerlo antes de continuar con este bloc de notas.

Puede entrenar modelos en Vertex AI con ayuda de AutoML o del entrenamiento personalizado. En el caso del entrenamiento personalizado, puede seleccionar muchos distintos tipos de máquinas para potenciar sus trabajos de entrenamiento, habilitar el entrenamiento distribuido, aplicar el ajuste de hiperparámetros y acelerar con GPU.

También puede atender solicitudes de predicción si se implementa el modelo entrenado en Vertex AI Models y se crea un punto de conexión.

En este tutorial, usaremos Vertex AI Training con trabajos personalizados para entrenar un modelo en una canalización de TFX. También implementaremos el modelo para atender solicitudes de predicción con ayuda de Vertex AI.


Este bloc de notas se diseñó para ejecutarse en [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb) o en [AI Platform Notebooks](https://cloud.google.com/ai-platform-notebooks). Si no usa ninguno de estos, simplemente puede hacer clic en el botón "Ejecutar en Google Colab" que se muestra más arriba.

## Preparación

Si completó el [Tutorial de canalizaciones simples de TFX para Vertex Pipelines](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple), tendrá un proyecto de GCP en funcionamiento y un cubo de GCS, y eso es todo lo que necesitamos para este tutorial. Lea primero el tutorial preliminar si aún no lo hizo.

### Instale paquetes de Python

Instalaremos los paquetes de Python necesarios, incluidos TFX y KFP, para crear canalizaciones de aprendizaje automático y enviar trabajos a canalizaciones de Vertex.

In [ ]:
# Use the latest version of pip.
!pip install --upgrade pip
!pip install --upgrade "tfx[kfp]<2"

### Desinstalación de shapely

TODO(b/263441833) Esta es una solución temporal para evitar un ImportError. En última instancia, debería solucionarse admitiendo una versión reciente de Bigquery, en lugar de desinstalar otras dependencias adicionales.

In [ ]:
!pip uninstall shapely -y

#### ¿Reinició el tiempo de ejecución?

Si está usando Google Colab, la primera vez que ejecute la celda anterior, debe hacer clic en el botón "RESTART RUNTIME" (REINICIAR TIEMPO DE EJECUCIÓN) o usar el menú "Tiempo de ejecución &gt; Reiniciar tiempo de ejecución ..." para reiniciar el tiempo de ejecución. Esto se debe a la forma en que Colab carga los paquetes.

Si no está en Colab, puede reiniciar el tiempo de ejecución con la siguiente celda.

In [ ]:
# docs_infra: no_execute
import sys
if not 'google.colab' in sys.modules:
  # Automatically restart kernel after installs
  import IPython
  app = IPython.Application.instance()
  app.kernel.do_shutdown(True)

### Inicie sesión en Google para este bloc de notas

Si está ejecutando este bloc de notas en Colab, autentíquese con su cuenta de usuario:

In [ ]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

**Si está en AI Platform Notebooks**, autentíquese con Google Cloud antes de ejecutar la siguiente sección, ejecutando lo siguiente

```sh
gcloud auth login
```

**en la ventana Terminal** (que puede abrir a través de **Archivo** &gt; **Nuevo** en el menú). Solo tiene que hacer esto una vez por instancia de bloc de notas.

Verifique las versiones del paquete.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import kfp
print('KFP version: {}'.format(kfp.__version__))

### Configuración de variables

Configuraremos algunas variables que se usan para personalizar las canalizaciones a continuación. Se requiere la siguiente información:

- Id del proyecto GCP. Consulte [Cómo identificar su id de proyecto](https://cloud.google.com/resource-manager/docs/creating-managing-projects#identifying_projects).
- Región de GCP para ejecutar canalizaciones. Para obtener más información sobre las regiones en las que las canalizaciones de Vertex están disponible, consulte la [guía de ubicaciones de Vertex AI](https://cloud.google.com/vertex-ai/docs/general/locations#feature-availability).
- Cubo de Google Cloud Storage para almacenar resultados de canalización.

**Ingrese los valores requeridos en la celda a continuación antes de ejecutarlo**.


In [ ]:
GOOGLE_CLOUD_PROJECT = ''     # <--- ENTER THIS
GOOGLE_CLOUD_REGION = ''      # <--- ENTER THIS
GCS_BUCKET_NAME = ''          # <--- ENTER THIS

if not (GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_REGION and GCS_BUCKET_NAME):
    from absl import logging
    logging.error('Please set all required parameters.')

Configure `gcloud` para usar su proyecto.

In [ ]:
!gcloud config set project {GOOGLE_CLOUD_PROJECT}

In [ ]:
PIPELINE_NAME = 'penguin-vertex-training'

# Path to various pipeline artifact.
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' Python module.
MODULE_ROOT = 'gs://{}/pipeline_module/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' data.
DATA_ROOT = 'gs://{}/data/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# Name of Vertex AI Endpoint.
ENDPOINT_NAME = 'prediction-' + PIPELINE_NAME

print('PIPELINE_ROOT: {}'.format(PIPELINE_ROOT))

### Cómo preparar datos de ejemplo

Usaremos el mismo [conjunto de datos Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/articles/intro.html) que el [Tutorial de canalizaciones simples de TFX](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

Hay cuatro características numéricas en este conjunto de datos que ya fueron normalizadas para tener un rango [0,1]. Compilaremos un modelo de clasificación que prediga las `species` de pingüinos.

Tenemos que hacer nuestra propia copia del conjunto de datos. Debido a que TFX ExampleGen lee entradas de un directorio, tenemos que crear un directorio y copiar el conjunto de datos en él.

In [ ]:
!gsutil cp gs://download.tensorflow.org/data/palmer_penguins/penguins_processed.csv {DATA_ROOT}/

Veamos rápidamente al archivo CSV.

In [ ]:
!gsutil cat {DATA_ROOT}/penguins_processed.csv | head

## Cómo crear una canalización

Nuestra canalización será muy similar a la canalización que creamos en el [Tutorial de canalizaciones simples de TFX para Vertex Pipelines](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple). La canalización constará de tres componentes, CsvExampleGen, Trainer y Pusher. Pero usaremos componentes Trainer y Pusher especiales. El componente Trainer trasladará las cargas de trabajo de entrenamiento a Vertex AI, y el componente Pusher publicará el modelo de ML entrenado en Vertex AI en lugar de hacerlo en un sistema de archivos.

TFX ofrece un `Trainer` especial para enviar trabajos de entrenamiento al servicio Vertex AI Training. Todo lo que tenemos que hacer es usar `Trainer` en el módulo de extensión en lugar del componente `Trainer` estándar junto con algunos parámetros GCP requeridos.

En este tutorial, ejecutaremos trabajos de entrenamiento de Vertex AI usando solo una CPU primero y luego una GPU.

TFX también proporciona un `Pusher` especial para cargar el modelo en *Vertex AI Models*. `Pusher` creará un recurso *Vertex AI Endpoint* para ofrecer predicciones en línea también. Consulte la [documentación de Vertex AI](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions) para obtener más información sobre las predicciones en línea proporcionadas por Vertex AI.

### Escriba el código del modelo

El modelo en sí es casi similar al modelo del [Tutorial de canalizaciones simples de TFX](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

Agregaremos la función `_get_distribution_strategy()` que crea una [estrategia de distribución de TensorFlow](https://www.tensorflow.org/guide/distributed_training) y se usa en `run_fn` para usar MirroredStrategy si la GPU está disponible.

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

# Copied from https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple and
# slightly modified run_fn() to add distribution_strategy.

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
        for feature in _FEATURE_KEYS
    }, _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _make_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# NEW: Read `use_gpu` from the custom_config of the Trainer.
#      if it uses GPU, enable MirroredStrategy.
def _get_distribution_strategy(fn_args: tfx.components.FnArgs):
  if fn_args.custom_config.get('use_gpu', False):
    logging.info('Using MirroredStrategy with one GPU.')
    return tf.distribute.MirroredStrategy(devices=['device:GPU:0'])
  return None


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  # NEW: If we have a distribution strategy, build a model in a strategy scope.
  strategy = _get_distribution_strategy(fn_args)
  if strategy is None:
    model = _make_keras_model()
  else:
    with strategy.scope():
      model = _make_keras_model()

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

Copie el archivo del módulo en GCS, al que se puede acceder desde los componentes de la canalización.

De lo contrario, es posible que sea conveniente compilar una imagen de contenedor que incluya el archivo del módulo y usar la imagen para ejecutar la canalización y los trabajos de AI Platform Training.

In [ ]:
!gsutil cp {_trainer_module_file} {MODULE_ROOT}/

### Cómo escribir una definición de canalización

Definiremos una función para crear una canalización de TFX. Tiene los mismos tres componentes que en el [Tutorial de canalizaciones simples de TFX](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple), pero usamos componentes `Trainer` y `Pusher` en el módulo de extensión de GCP.

`tfx.extensions.google_cloud_ai_platform.Trainer` se comporta como un `Trainer` normal, pero simplemente mueve el cálculo para el entrenamiento del modelo a la nube. Inicia un trabajo personalizado en el servicio Vertex AI Training y el componente de entrenador en el sistema de orquestación se limitará a esperar que se complete el trabajo de Vertex AI Training.

`tfx.extensions.google_cloud_ai_platform.Pusher` crea un modelo Vertex AI y un punto de conexión Vertex AI a partir del modelo entrenado.


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, endpoint_name: str, project_id: str,
                     region: str, use_gpu: bool) -> tfx.dsl.Pipeline:
  """Implements the penguin pipeline with TFX."""
  # Brings data into the pipeline or otherwise joins/converts training data.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # NEW: Configuration for Vertex AI Training.
  # This dictionary will be passed as `CustomJobSpec`.
  vertex_job_spec = {
      'project': project_id,
      'worker_pool_specs': [{
          'machine_spec': {
              'machine_type': 'n1-standard-4',
          },
          'replica_count': 1,
          'container_spec': {
              'image_uri': 'gcr.io/tfx-oss-public/tfx:{}'.format(tfx.__version__),
          },
      }],
  }
  if use_gpu:
    # See https://cloud.google.com/vertex-ai/docs/reference/rest/v1/MachineSpec#acceleratortype
    # for available machine types.
    vertex_job_spec['worker_pool_specs'][0]['machine_spec'].update({
        'accelerator_type': 'NVIDIA_TESLA_K80',
        'accelerator_count': 1
    })

  # Trains a model using Vertex AI Training.
  # NEW: We need to specify a Trainer for GCP with related configs.
  trainer = tfx.extensions.google_cloud_ai_platform.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5),
      custom_config={
          tfx.extensions.google_cloud_ai_platform.ENABLE_VERTEX_KEY:
              True,
          tfx.extensions.google_cloud_ai_platform.VERTEX_REGION_KEY:
              region,
          tfx.extensions.google_cloud_ai_platform.TRAINING_ARGS_KEY:
              vertex_job_spec,
          'use_gpu':
              use_gpu,
      })

  # NEW: Configuration for pusher.
  vertex_serving_spec = {
      'project_id': project_id,
      'endpoint_name': endpoint_name,
      # Remaining argument is passed to aiplatform.Model.deploy()
      # See https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api#deploy_the_model
      # for the detail.
      #
      # Machine type is the compute resource to serve prediction requests.
      # See https://cloud.google.com/vertex-ai/docs/predictions/configure-compute#machine-types
      # for available machine types and acccerators.
      'machine_type': 'n1-standard-4',
  }

  # Vertex AI provides pre-built containers with various configurations for
  # serving.
  # See https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers
  # for available container images.
  serving_image = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-6:latest'
  if use_gpu:
    vertex_serving_spec.update({
        'accelerator_type': 'NVIDIA_TESLA_K80',
        'accelerator_count': 1
    })
    serving_image = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.2-6:latest'

  # NEW: Pushes the model to Vertex AI.
  pusher = tfx.extensions.google_cloud_ai_platform.Pusher(
      model=trainer.outputs['model'],
      custom_config={
          tfx.extensions.google_cloud_ai_platform.ENABLE_VERTEX_KEY:
              True,
          tfx.extensions.google_cloud_ai_platform.VERTEX_REGION_KEY:
              region,
          tfx.extensions.google_cloud_ai_platform.VERTEX_CONTAINER_IMAGE_URI_KEY:
              serving_image,
          tfx.extensions.google_cloud_ai_platform.SERVING_ARGS_KEY:
            vertex_serving_spec,
      })

  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=components)

## Ejecute la canalización en canalizaciones de Vertex

Usaremos Vertex Pipelines para ejecutar la canalización como lo hicimos en el [Tutorial de canalizaciones simples de TFX para Vertex Pipelines](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple).

In [ ]:
# docs_infra: no_execute
import os

PIPELINE_DEFINITION_FILE = PIPELINE_NAME + '_pipeline.json'

runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
    config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(),
    output_filename=PIPELINE_DEFINITION_FILE)
_ = runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        endpoint_name=ENDPOINT_NAME,
        project_id=GOOGLE_CLOUD_PROJECT,
        region=GOOGLE_CLOUD_REGION,
        # We will use CPUs only for now.
        use_gpu=False))

El archivo de definición generado se puede enviar utilizando el cliente aiplatform de Google Cloud en el paquete `google-cloud-aiplatform`.

In [ ]:
# docs_infra: no_execute
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
import logging
logging.getLogger().setLevel(logging.INFO)

aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=GOOGLE_CLOUD_REGION)

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.submit()

Ahora puede visitar el enlace en el resultado anterior o visitar 'Vertex AI &gt; Pipelines' en [Google Cloud Console](https://console.cloud.google.com/) para ver el progreso.

## Prueba con una solicitud de predicción

Una vez que se complete la canalización, encontrará un modelo *implementado* en uno de los puntos de conexión en 'Vertex AI &gt; Puntos de conexión'. Necesitamos conocer la identificación del punto de conexión para enviar una solicitud de predicción al nuevo punto de conexión. Esto es diferente del *nombre del punto de conexión* que ingresamos anteriormente. Puede encontrar la identificación en la [página Puntos de conexión](https://console.cloud.google.com/vertex-ai/endpoints) en `Google Cloud Console`; parece un número muy largo.

**Establezca ENDPOINT_ID a continuación antes de ejecutarlo.**


In [ ]:
ENDPOINT_ID=''     # <--- ENTER THIS
if not ENDPOINT_ID:
    from absl import logging
    logging.error('Please set the endpoint id.')

Usamos el mismo cliente aiplatform para enviar una solicitud al punto de conexión. Enviaremos una solicitud de predicción para la clasificación de especies de pingüinos. La entrada son las cuatro características que utilizamos y el modelo devolverá tres valores, porque nuestro modelo genera un valor para cada especie.

Por ejemplo, el siguiente ejemplo específico tiene el valor más grande en el índice '2' e imprimirá '2'.


In [ ]:
# docs_infra: no_execute
import numpy as np

# The AI Platform services require regional API endpoints.
client_options = {
    'api_endpoint': GOOGLE_CLOUD_REGION + '-aiplatform.googleapis.com'
    }
# Initialize client that will be used to create and send requests.
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

# Set data values for the prediction request.
# Our model expects 4 feature inputs and produces 3 output values for each
# species. Note that the output is logit value rather than probabilities.
# See the model code to understand input / output structure.
instances = [{
    'culmen_length_mm':[0.71],
    'culmen_depth_mm':[0.38],
    'flipper_length_mm':[0.98],
    'body_mass_g': [0.78],
}]

endpoint = client.endpoint_path(
    project=GOOGLE_CLOUD_PROJECT,
    location=GOOGLE_CLOUD_REGION,
    endpoint=ENDPOINT_ID,
)
# Send a prediction request and get response.
response = client.predict(endpoint=endpoint, instances=instances)

# Uses argmax to find the index of the maximum value.
print('species:', np.argmax(response.predictions[0]))

Para obtener información detallada sobre la predicción en línea, visite la [página Puntos de conexión](https://console.cloud.google.com/vertex-ai/endpoints) en `Google Cloud Console`. Puede encontrar una guía sobre cómo enviar solicitudes de muestra y enlaces a más recursos.

## Ejecute la canalización usando una GPU


Vertex AI admite el entrenamiento utilizando varios tipos de máquinas, incluida la compatibilidad con GPU. Consulte la [referencia de especificaciones de la máquina](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/MachineSpec#acceleratortype) para conocer las opciones disponibles.

Ya definimos nuestra canalización para respaldar el entrenamiento en GPU. Todo lo que tenemos que hacer es configurar la marca `use_gpu` en True. Luego, se creará una canalización con una especificación de máquina que incluye una NVIDIA_TESLA_K80 y nuestro código de entrenamiento de modelo usará `tf.distribute.MirroredStrategy`.

Tenga en cuenta que la marca `use_gpu` no forma parte de la API de Vertex o TFX. Solo se usa para controlar el código de entrenamiento en este tutorial.

In [ ]:
# docs_infra: no_execute
runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        endpoint_name=ENDPOINT_NAME,
        project_id=GOOGLE_CLOUD_PROJECT,
        region=GOOGLE_CLOUD_REGION,
        # Updated: Use GPUs. We will use a NVIDIA_TESLA_K80 and 
        # the model code will use tf.distribute.MirroredStrategy.
        use_gpu=True))

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.submit()

Ahora puede visitar el enlace en el resultado anterior o visitar 'Vertex AI &gt; Canalizaciones' en [Google Cloud Console](https://console.cloud.google.com/) para ver el progreso.

## Limpieza

Ha creado un modelo y un punto de conexión de Vertex AI en este tutorial. Elimine estos recursos para evitar cargos no deseados. Para ello, vaya a [Puntos de conexión](https://console.cloud.google.com/vertex-ai/endpoints) y *anule la implementación* del modelo desde el punto de conexión. Luego puede eliminar el punto de conexión y el modelo por separado.